# XGB Models

The Data will be split by week and use week 9 as a validation set 

The Workflow will be as follow: 

    ** Use the default xgbregresor to train and plot feature importance
    ** Use a gridsearch to find the ideal hyperparameters (with only 50 estimators) 
    ** Train using all data and 500 estimators as our final model 
    

## Imports 

In [1]:
import os

mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'

os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

import xgboost as xgb

C:\Anaconda\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
import pandas as pd
import numpy as np
import re 
import os
import time
from IPython.display import display
import matplotlib.pyplot as plt
from sklearn.cross_validation import LabelKFold
from sklearn.grid_search import GridSearchCV
import seaborn as sns
%matplotlib inline

## Util functions

In [8]:
# Load computed Dataset
def load_dataset(folder = ''):
    
    
    filename = "jorge_dataset.csv"
    filepath = os.path.join(folder,filename)        
    
    df_train = pd.read_csv(filepath,
                           usecols = ['Semana','Producto_ID', 'Cliente_ID', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'ZipCode',
                                      'Mean2','Mean3','Mean4','Mean6','Mean7',
                                      'Median2','Median3','Median4','Median6','Median7',
                                      'Last_per_Agencia_ID', 'Last_per_Canal_ID','Last_per_Ruta_SAK','Last_per_Cliente_ID',
                                      'Last_per_ZipCode','id',
                                      'Demanda_uni_equil'],
                           dtype  = {'Producto_ID':'int32',
                                     'Semana':'int8',
                                     'Cliente_ID':'int32',
                                     'Agencia_ID':'uint16',
                                     'Canal_ID':'int8',
                                     'Ruta_SAK':'int32',
                                     'ZipCode':'int16',                                     
                                     'Mean2':'float32','Mean3':'float32','Mean4':'float32',
                                     'Mean6':'float32','Mean7':'float32',
                                     'Median2':'float32','Median3':'float32','Median4':'float32',
                                     'Median6':'float32','Median7':'float32',
                                     'Last_per_Ruta_SAK':'float32','Last_per_Cliente_ID':'float32',
                                     'Demanda_uni_equil':'int32'})
    return df_train

## Load Dataset 

In [9]:
df = load_dataset()

ValueError: 'Cliente_ID' is not in list

In [5]:
print df.columns
display(df.head())
print df.dtypes

NameError: name 'df' is not defined